Retention – один из самых важных показателей в компании. Ваша задача – написать функцию, которая будет считать retention игроков (по дням от даты регистрации игрока). Функция доожна быть тестируема как на полном датасете, так и на сэмпле данных.

In [3]:
def retention(N_day, start_date = None, end_date = None, cohort = None, percentage=True):
    """
    Функция retention может возвращать несколько значений:
    - Среднее процентное значение Retention конкретного дня по всем когортам.
    - Процентное значение Retention конкретного дня по конкретной когорте.
    - Абсолютное среднее значение Retention конкретного дня по всем когортам.
    - Абсолютное значение Retention конкретного дня по конкретной когорте.
    
    Параметры:  
    - N_day - Retention какого дня требуется
    - start_date - начало регистрации пользователей. По-умолчанию min дата, формат: 'Y-m-d'
    - end_date - конец регистрации пользователей. По-умолчанию max дата, формат: 'Y-m-d'
    - cohort - Retention какой именно когорты пользователей требуется. По-умолчанию все когорты, формат: 'Y-m-d'
    - percentage - Retention в процентах или в абсолютном выражении. По-умолчанию True, формат: True/False
            
    """
           
    import pandas as pd
    
    # преобразование start_date и end_date в даты
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # чтение данных о регистрациях с преобразванием дат 
    reg_data = pd.read_csv('problem1-reg_data.csv', sep=';')
    reg_data.reg_ts = (pd.to_datetime(reg_data.reg_ts, unit='s')).dt.date
    
    #ограничение датасета временными рамками start_date, end_date
    if start_date == None: start_date = reg_data.reg_ts.min()
    if end_date == None: end_date = reg_data.reg_ts.max()
    reg_data = reg_data.query('reg_ts >= @start_date & reg_ts <= @end_date')
    
    # чтение данных о посещениях с преобразованием дат
    auth_data = pd.read_csv('problem1-auth_data.csv', sep=';')
    auth_data.auth_ts = (pd.to_datetime(auth_data.auth_ts, unit='s')).dt.date
    
    # объединение датасетов
    df = auth_data.merge(reg_data, how='right', on='uid')
    
    # определяем временные промежутки для построения Retention
    df['days_distance'] = (df.auth_ts - df.reg_ts).dt.days
    
    # определяем когорты пользователей по дате регитрации
    df['cohorts'] = df.reg_ts.apply(lambda x: str(x))
    
    # групируем данные для формирования сводной таблицы
    retention = df.groupby(['cohorts', 'days_distance'], as_index=False).uid.nunique()
    
    # если требуется абсолютный Retention
    if percentage == False:
        retention = retention.pivot_table(index='cohorts', columns='days_distance', values='uid').reset_index()
        if cohort == None:
            return print('Абсолютный средний Retention {}-го дня за период с {} по {} составляет: {:.2f}'.format(N_day, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d') ,(retention.loc[:, N_day].mean())))
        else:
            return print('Абсолютный Retention {}-го дня для когорты {} за период с {} по {} составляет: {:.2f}'.format(N_day, cohort, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d') ,retention[retention.cohorts == cohort].loc[:,N_day].sum()))
        
        
    # еще раз группируем данные и считем процентное значение Retention
    group_df = retention.query('days_distance == 0')[['cohorts', 'uid']].rename(columns={'uid' : 'total_users'})
    retention = retention.merge(group_df, how='left', on='cohorts')
    retention['retention_values'] = round(((retention.uid / retention.total_users)), 4)
    
    # итоговая таблица с посчитанным процентным Retention
    retention = retention.pivot_table(index='cohorts', columns='days_distance', values='retention_values').reset_index()
    
    # выводим результат
    if cohort == None:
        return print('Retention {}-го дня за период с {} по {} составляет: {:.2%}'.format(N_day, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d') ,(retention.loc[:, N_day].mean())))
    else:
        return print('Retention {}-го дня для когорты {} за период с {} по {} составляет: {:.2%}'.format(N_day, cohort, start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d') ,retention[retention.cohorts == cohort].loc[:,N_day].sum()))

In [4]:
retention(N_day=19, start_date='2020-04-05')

Retention 19-го дня за период с 2020-04-05 по 2020-09-23 составляет: 4.12%
